In [1]:
from crewai import Agent, Task, Crew, Process
from openai import OpenAI
from dotenv import load_dotenv
from datetime import datetime
import os

load_dotenv()  # Load environment variables from .env
COMPLETION_PHRASE = "No major issues found."
MODEL = "gpt-4o-mini"   # or "gpt-4-turbo" / Gemini equivalent if configured via Vertex adapter

# --- Shared client (optional) ---
client = OpenAI()


In [3]:
"""
Story Improvement Loop (CrewAI)
-------------------------------

This script demonstrates a simple Agentic AI workflow:

1. **Initial Writer Agent** creates a short draft.
2. **Critic Agent** evaluates it and provides improvement suggestions.
3. **Refiner Agent** improves the story using that feedback.
4. Steps 2–3 repeat in a loop until the critic says the story is “good enough”.

This teaches beginners the core pattern of:
 → Generate → Evaluate → Refine (iterative agents loop)

Everything is intentionally simple and well-commented.
"""

# ===============================
# 🚀 IMPORTS & SETUP
# ===============================
from crewai import Agent, Task, Crew, Process
from crewai import LLM
from dotenv import load_dotenv

load_dotenv()

# This phrase will tell the system to stop the loop when quality is sufficient.
COMPLETION_PHRASE = "NO_FURTHER_IMPROVEMENT_NEEDED"

# Use any model available (OpenAI or local)
llm = LLM(model="gpt-4o-mini")


# ===============================
# 🤖 DEFINE AGENTS
# ===============================

# 1️⃣ Initial Writer Agent
initial_writer = Agent(
    role="InitialStoryWriter",
    goal="Write a simple and clear first draft of a story topic.",
    backstory="You enjoy writing short, imaginative story seeds.",
    verbose=True,
    llm=llm
)

# 2️⃣ Critic Agent
critic = Agent(
    role="StoryCritic",
    goal="Evaluate story clarity, engagement, and writing quality.",
    backstory="You carefully review writing and suggest small improvements.",
    verbose=True,
    llm=llm
)

# 3️⃣ Refiner Agent
refiner = Agent(
    role="StoryRefiner",
    goal="Improve story drafts using feedback from the critic.",
    backstory="You rewrite stories without changing their core idea.",
    verbose=True,
    llm=llm
)


# ===============================
# 📌 STORY TOPIC
# ===============================
topic = "A child finds a speaking tree in their backyard."


# ===============================
# 📝 TASK DEFINITIONS (ROUND 1)
# ===============================

# Task 1: Initial draft
initial_draft = Task(
    description=f"""
Write the *first draft* of a short story (2–4 sentences) on the topic:
"{topic}".

REQUIREMENTS:
- Introduce a character, a setting, or an action clearly.
- Keep it simple and easy to refine later.
- Output **only** the story text, nothing else.
""",
    expected_output="A 2–4 sentence story draft.",
    agent=initial_writer
)


# ===============================
# 🧠 BUILD CREW FOR FIRST DRAFT
# ===============================
crew = Crew(
    agents=[initial_writer, critic, refiner],
    tasks=[initial_draft],
    process=Process.sequential
)

print("\n🧠 Starting CrewAI Story Drafting Loop...\n")

# Step 1: Run the first draft
draft_output = crew.kickoff()
current_story = draft_output.raw  # extract story text safely

print("📄 Initial Draft:\n", current_story)


# ===============================
# 🔁 ITERATIVE REFINEMENT LOOP
# ===============================
MAX_ITERS = 5
print("\n🔄 Entering refinement loop...\n")

for iteration in range(1, MAX_ITERS + 1):
    print(f"\n====== Iteration {iteration} ======\n")

    # 1️⃣ Task: Critic evaluates the current story
    critique_task = Task(
        description=f"""
You are reviewing the following story:

\"\"\"{current_story}\"\"\"

Your job:
- Provide 1–2 short suggestions to improve clarity, pacing, or engagement.
- If the story is already polished and needs no improvement,
  respond EXACTLY with this phrase:

{COMPLETION_PHRASE}

RULES:
- Keep feedback short (1–2 bullets).
- DO NOT rewrite the story.
- DO NOT include compliments.
""",
        expected_output=f"Critique text OR the exact phrase '{COMPLETION_PHRASE}'.",
        agent=critic
    )

    # 2️⃣ Task: Refiner improves story if needed
    refine_task = Task(
        description=f"""
You are improving the story based on the critique.

STORY:
\"\"\"{current_story}\"\"\"

CRITIQUE:
This is the critique output from the critic agent.

INSTRUCTIONS:
- If the critique equals '{COMPLETION_PHRASE}', output exactly this signal:
  '{COMPLETION_PHRASE}'
- Otherwise:
  Rewrite the story using the critique.
  Preserve meaning, tone, and structure.
  Output **only** the improved story text.
""",
        expected_output="Revised story OR the exact completion phrase.",
        agent=refiner
    )

    # 3️⃣ Build a mini-crew for this loop iteration
    loop_crew = Crew(
        agents=[critic, refiner],
        tasks=[critique_task, refine_task],
        process=Process.sequential
    )

    loop_output = loop_crew.kickoff()

    loop_raw = loop_output.raw  # safest extraction

    # Stop if refinement not needed
    if COMPLETION_PHRASE in loop_raw:
        print("\n✅ Story judged complete. Exiting loop.\n")
        break

    # Otherwise update story
    current_story = loop_raw
    print("📄 Updated Story:\n", current_story)


# ===============================
# 🎉 FINAL RESULT
# ===============================
print("\n============================")
print("📚 FINAL STORY OUTPUT:")
print("============================\n")
print(current_story)



🧠 Starting CrewAI Story Drafting Loop...



╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: InitialStoryWriter                                                                                      │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Write the *first draft* of a short story (2–4 sentences) on the topic:                                         │
│  "A child finds a speaking tree in their backyard.".                                                            │
│                                                                                                                 │
│  REQUIREMENTS:                                                                                                  │
│  - Introduce a character, a setting, or an action clearly.                                                      │
│  - Keep it simple and easy to refine later.                                                                     │
│  - Output **only** the story text, nothing else.                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: InitialStoryWriter                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Little Mia was playing in her backyard when she stumbled upon an ancient oak tree with gnarled roots and       │
│  shimmering leaves. To her surprise, the tree spoke in a gentle, rustling whisper, sharing tales of the world   │
│  it had seen grow around it over the centuries. With wide eyes, Mia climbed up and nestled in its branches,     │
│  eager to listen to the stories of magic and adventure. From that day on, every afternoon was an enchanting     │
│  journey through time, filled with wisdom and wonder.                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

📄 Initial Draft:
 Little Mia was playing in her backyard when she stumbled upon an ancient oak tree with gnarled roots and shimmering leaves. To her surprise, the tree spoke in a gentle, rustling whisper, sharing tales of the world it had seen grow around it over the centuries. With wide eyes, Mia climbed up and nestled in its branches, eager to listen to the stories of magic and adventure. From that day on, every afternoon was an enchanting journey through time, filled with wisdom and wonder.

🔄 Entering refinement loop...


====== Iteration 1 ======



╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: StoryCritic                                                                                             │
│                                                                                                                 │
│  Task:                                                                                                          │
│  You are reviewing the following story:                                                                         │
│                                                                                                                 │
│  """Little Mia was playing in her backyard when she stumbled upon an ancient oak tree with gnarled roots and    │
│  shimmering leaves. To her surprise, the tree spoke in a gentle, rustling whisper, sharing tales of the world   │
│  it had seen grow around it over the centuries. With wide eyes, Mia climbed up and nestled in its branches,     │
│  eager to listen to the stories of magic and adventure. From that day on, every afternoon was an enchanting     │
│  journey through time, filled with wisdom and wonder."""                                                        │
│                                                                                                                 │
│  Your job:                                                                                                      │
│  - Provide 1–2 short suggestions to improve clarity, pacing, or engagement.                                     │
│  - If the story is already polished and needs no improvement,                                                   │
│    respond EXACTLY with this phrase:                                                                            │
│                                                                                                                 │
│  NO_FURTHER_IMPROVEMENT_NEEDED                                                                                  │
│                                                                                                                 │
│  RULES:                                                                                                         │
│  - Keep feedback short (1–2 bullets).                                                                           │
│  - DO NOT rewrite the story.                                                                                    │
│  - DO NOT include compliments.                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: StoryCritic                                                                                             │
│                                                                                                                 │
│  Task:                                                                                                          │
│  You are reviewing the following story:                                                                         │
│                                                                                                                 │
│  """Little Mia was playing in her backyard when she stumbled upon an ancient oak tree. Its gnarled roots        │
│  twisted like ancient fingers, and its shimmering leaves danced in the sunlight. To her surprise, the tree      │
│  spoke in a gentle, rustling whisper. It shared tales of the world it had watched grow around it over the       │
│  centuries.                                                                                                     │
│                                                                                                                 │
│  One afternoon, the tree recounted the story of a young knight who bravely saved a village from a fierce        │
│  dragon. As it spoke, Mia could almost see the vibrant colors of the knight’s armor glimmering against the      │
│  backdrop of a fiery sky. The tale taught Mia the importance of courage and kindness.                           │
│                                                                                                                 │
│  With wide eyes and a heart racing with excitement, Mia climbed up and nestled in its sturdy branches, eager    │
│  to listen to more stories of magic and adventure. The sweet scent of blooming flowers wafted through the air,  │
│  adding to the enchantment of the moment. As the weaver of these captivating tales, the tree reminded her that  │
│  every story holds a lesson for those willing to listen.                                                        │
│                                                                                                                 │
│  After each session, Mia felt a spark of inspiration and a deep warmth in her heart. She realized that these    │
│  stories weren't just tales but precious gifts that would guide her on her own journey. From that day on,       │
│  every afternoon became an enchanting voyage through time, filled with wisdom and wonder, connecting Mia ever   │
│  more deeply to the magic of storytelling."""                                                                   │
│                                                                                                                 │
│  Your job:                                                                                                      │
│  - Provide 1–2 short suggestions to improve clarity, pacing, or engagement.                                     │
│  - If the story is already polished and needs no improvement,                                                   │
│    respond EXACTLY with this phrase:                                                                            │
│                                                                                                                 │
│  NO_FURTHER_IMPROVEMENT_NEEDED                                                                                  │
│                                                                                                                 │
│  RULES:                                                                                                         │
│  - Keep feedback short (1–2 bullets).                  

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: StoryCritic                                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  NO_FURTHER_IMPROVEMENT_NEEDED                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: StoryRefiner                                                                                            │
│                                                                                                                 │
│  Task:                                                                                                          │
│  You are improving the story based on the critique.                                                             │
│                                                                                                                 │
│  STORY:                                                                                                         │
│  """Little Mia was playing in her backyard when she stumbled upon an ancient oak tree. Its gnarled roots        │
│  twisted like ancient fingers, and its shimmering leaves danced in the sunlight. To her surprise, the tree      │
│  spoke in a gentle, rustling whisper. It shared tales of the world it had watched grow around it over the       │
│  centuries.                                                                                                     │
│                                                                                                                 │
│  One afternoon, the tree recounted the story of a young knight who bravely saved a village from a fierce        │
│  dragon. As it spoke, Mia could almost see the vibrant colors of the knight’s armor glimmering against the      │
│  backdrop of a fiery sky. The tale taught Mia the importance of courage and kindness.                           │
│                                                                                                                 │
│  With wide eyes and a heart racing with excitement, Mia climbed up and nestled in its sturdy branches, eager    │
│  to listen to more stories of magic and adventure. The sweet scent of blooming flowers wafted through the air,  │
│  adding to the enchantment of the moment. As the weaver of these captivating tales, the tree reminded her that  │
│  every story holds a lesson for those willing to listen.                                                        │
│                                                                                                                 │
│  After each session, Mia felt a spark of inspiration and a deep warmth in her heart. She realized that these    │
│  stories weren't just tales but precious gifts that would guide her on her own journey. From that day on,       │
│  every afternoon became an enchanting voyage through time, filled with wisdom and wonder, connecting Mia ever   │
│  more deeply to the magic of storytelling."""                                                                   │
│                                                                                                                 │
│  CRITIQUE:                                                                                                      │
│  This is the critique output from the critic agent.                                                             │
│                                                                                                                 │
│  INSTRUCTIONS:                                                                                                  │
│  - If the critique equals 'NO_FURTHER_IMPROVEMENT_NEEDED', output exactly this signal:                          │
│    'NO_FURTHER_IMPROVEMENT_NEEDED'                                                                              │
│  - Otherwise:                                                                                                   │
│    Rewrite the story using the critique.               

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: StoryRefiner                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  NO_FURTHER_IMPROVEMENT_NEEDED                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


✅ Story judged complete. Exiting loop.


📚 FINAL STORY OUTPUT:

Little Mia was playing in her backyard when she stumbled upon an ancient oak tree. Its gnarled roots twisted like ancient fingers, and its shimmering leaves danced in the sunlight. To her surprise, the tree spoke in a gentle, rustling whisper. It shared tales of the world it had watched grow around it over the centuries. 

One afternoon, the tree recounted the story of a young knight who bravely saved a village from a fierce dragon. As it spoke, Mia could almost see the vibrant colors of the knight’s armor glimmering against the backdrop of a fiery sky. The tale taught Mia the importance of courage and kindness. 

With wide eyes and a heart racing with excitement, Mia climbed up and nestled in its sturdy branches, eager to listen to more stories of magic and adventure. The sweet scent of blooming flowers wafted through the air, adding to the enchantment of the moment. As the weaver of these captivating tales, the tree rem